<a href="https://colab.research.google.com/github/LeonSagebrand/Neural-Network/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (B)

import numpy as np

def sigmoid(x: np.array) -> np.array:
    return 1 / (1 + np.exp(-x))

class NeuronLayer:
    def __init__(self, num_inputs: int, num_neurons: int):
        self.weights = 2 * np.random.rand(num_inputs, num_neurons) - 1
        self.biases = 2*np.random.rand(num_neurons) - 1

    def forward(self, input_vector:np.array) -> np.array:
        weighted_sum = np.dot(input_vector, self.weights) + self.biases
        return sigmoid(weighted_sum)

num_inputs = 3
num_neurons = 4
layer = NeuronLayer(num_inputs, num_neurons)

input_vector = np.array([0.5, 0.3, 0.2])

output = layer.forward(input_vector)
print("Output:")
print(output)

Output:
[0.18004632 0.6132561  0.51156344 0.22626544]


In [ ]:
# (C)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define the neural network model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return F.log_softmax(x, dim=1)

# Initialize the model
model = MLP()

data = torch.randn(64, 784)
targets = torch.randint(0, 10, (64,))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10): # Loop the epochs
    optimizer.zero_grad()  #Clear previous gradients
    output = model(data)  # Forward pass
    loss = criterion(output, targets) # Calculate loss
    loss.backward() # Backpropagation
    optimizer.step()  # Update weights using ADAM
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


In [ ]:
# (D)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Check and list all available CUDA-GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")

# Simple neural network model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x

# Initialize the model and move it to GPU if its available
model = MLP().to(device)

# Generate dummy data and move to GPU
data = torch.randn(64, 784).to(device)
targets = torch.randint(0, 10, (64,)).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, targets)
    loss.backward()  # Backprop
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

CUDA is not available.
Training on device: cpu
Epoch 1, Loss: 2.3594777584075928
Epoch 2, Loss: 2.226069450378418
Epoch 3, Loss: 2.1218550205230713
Epoch 4, Loss: 2.0327858924865723
Epoch 5, Loss: 1.9484694004058838
Epoch 6, Loss: 1.8616819381713867
Epoch 7, Loss: 1.7699475288391113
Epoch 8, Loss: 1.6733680963516235
Epoch 9, Loss: 1.5706124305725098
Epoch 10, Loss: 1.4618173837661743


In [ ]:
# D with training data

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Added import for datasets and transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader  # Added import for DataLoader

# Check and list all available CUDA-GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

# Select the first CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")

# Simple neural network model


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x


# Initialize the model and move it to GPU if its available
model = MLP().to(device)

# Added transformaions and dataset loading
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to [0.0, 1.0]
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images
])

# Load the MNIST dataset
train_set = datasets.MNIST(root='./data', train=True,
                           download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False,
                          download=True, transform=transform)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(10):  # Loop over epochs
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:  # Replace with real data
        images, labels = images.view(
            images.shape[0], -1).to(device), labels.to(device)
        optimizer.zero_grad()  # Clear previous gradients
        output = model(images)  # Forward pass
        loss = criterion(output, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights using ADAM
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:  # Use test_loader for evaluation
        images, labels = images.view(
            images.shape[0], -1).to(device), labels.to(device)
        output = model(images)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

CUDA is not available.
Training on device: cpu
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15950824.06it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 479094.51it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4358265.23it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4887257.25it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.3871434788936491
Epoch 2, Loss: 0.18783529123851359
Epoch 3, Loss: 0.13934205190391777
Epoch 4, Loss: 0.11497068758716365
Epoch 5, Loss: 0.09843727363360875
Epoch 6, Loss: 0.08588445664861047
Epoch 7, Loss: 0.07589111595897616
Epoch 8, Loss: 0.06959962990218853
Epoch 9, Loss: 0.06281482082442132
Epoch 10, Loss: 0.056169756100143686
Test Accuracy: 96.97%


In [ ]:
# Del 2

# Improvements for the perceptron:

#     1. Another layer added. A third layer and Convolutional layers added
#     2. Adding BatchNorm1d for stabilized training.
#     3. Applied a dropout layer after each hidden layer which helps reduce overfitting.
#     4. Weight decay parameter set to 0.01 which reduces overfitting.
#     5. Learning rate schedule added to reduce learning rate over time, to improve convergence.
#     6. The training stops if validation loss is not improved for 5 consecutive epochs. Prevents overfitting. A new class added for this.
#     7. Normalized. Standardized input data
#     8. Applied horizontal flipping and random rotation

In [ ]:
# I tried shortening down the code for this model
# With this code the loss was improved significantly and overall a decreasing loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

class EarlyStopping:
    # Class for stopping training if the loss does not improve/decrease for 5 epochs
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class ImprovedCNN(nn.Module):
    #Improved CNN model with additional layers and dropout for regularization
    #
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.25),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return F.log_softmax(self.classifier(x), dim=1)

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Transformations for data augmentation and normalization
    transform = transforms.Compose([
        transforms.Resize(28),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomCrop(24),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load dataset
    dataset = datasets.MNIST('.', train=True, transform=transform, download=True)
    train_data, val_data = random_split(dataset, [int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))])
    train_loader = DataLoader(train_data, 64, shuffle=True)
    val_loader = DataLoader(val_data, 64)


    # Setup the model, optimizer, scheduler, and early stopping mechanism
    model = ImprovedCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = StepLR(optimizer, 5, 0.5)
    early_stopping = EarlyStopping()



    # Training loop with early stopping
    for epoch in range(30):
        model.train()
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            loss = F.cross_entropy(model(data), targets)
            loss.backward()
            optimizer.step()

        scheduler.step()
        val_loss = 0
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            output = model(data)
            val_loss += F.cross_entropy(output, targets).item()
        val_loss /= len(val_loader)

        print(f'Epoch {epoch + 1}, Val Loss: {val_loss:.4f}')
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

if __name__ == '__main__':
    main()


Epoch 1, Val Loss: 0.1875
Epoch 2, Val Loss: 0.1592
Epoch 3, Val Loss: 0.1504
Epoch 4, Val Loss: 0.1436
Epoch 5, Val Loss: 0.1439
Epoch 6, Val Loss: 0.1218


KeyboardInterrupt: 

In [ ]:
# Del 3

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50, ResNet50_Weights

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class TransferCNN(nn.Module):
    def __init__(self):
        super(TransferCNN, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Transformations for preprocessing images for ResNet50
    # I found the Resnet resolution is typically 224x224. Resize to 256 pixels
    # and the use CenterCrop for 224 pixels to avoid distortion.
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load datasets
    train_dataset = datasets.ImageFolder('/content/archive/archive/train', transform=transform)
    val_dataset = datasets.ImageFolder('/content/archive/archive/val', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)


    model = TransferCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    criterion = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping()

    for epoch in range(10): # Since we are using a pre-trained model I reduced the amount of epochs here
        model.train()
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        scheduler.step()
        val_loss = 0
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)

        print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Val Loss: {val_loss:.4f}')
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


Epoch 1, Loss: 0.48482638597488403, Val Loss: 0.5107
Epoch 2, Loss: 0.36713695526123047, Val Loss: 0.4728
Epoch 3, Loss: 0.16966404020786285, Val Loss: 0.4861
Epoch 4, Loss: 0.14115053415298462, Val Loss: 0.4676
Epoch 5, Loss: 0.04118746146559715, Val Loss: 0.4798
Epoch 6, Loss: 0.07436146587133408, Val Loss: 0.4645
Epoch 7, Loss: 0.023006947711110115, Val Loss: 0.4656
Epoch 8, Loss: 0.05097193643450737, Val Loss: 0.4746
Epoch 9, Loss: 0.061101034283638, Val Loss: 0.4906
Epoch 10, Loss: 0.08362790197134018, Val Loss: 0.4657


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os

# Assuming the zip file is named 'archive.zip'
zip_file = 'archive.zip'
extract_path = '/content/archive'

# Create the directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Extraction complete.


In [ ]:
import os

def find_classes(directory):
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Directory {directory} does not exist.")
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
    if not classes:
        raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")
    return classes

if __name__ == '__main__':
    # Path to the extracted folder
    directory = '/content/extracted_folder/train'
    try:
        classes = find_classes(directory)
        print("Classes found:", classes)
    except FileNotFoundError as e:
        print(e)


Directory /content/extracted_folder/train does not exist.
